# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [312]:
import pandas as pd
import requests
import json

In [313]:
# Setting up the config for usage.
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    username = config['username']
    print("Config Loaded Successfully.")
    key_list = [key1, key2, key3]
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    username = input('Enter your username: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
#     with open('config.json.sample','r') as f:
#         config = json.load(f)
    config = {'user_agent':None,'key1':None, 'key2':None, 'key3':None, 'username':None}
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    config['username'] = username
    
    with open('config.json','w+') as f:
        json.dump(config, f)
    
    key_list = [key1, key2, key3]

    print()
    print("Your configuration has been saved to config.json!")
    
# Fetches variables: user_agent, username, key1, key2, key3, key_list

Config Loaded Successfully.


# Fetching Data from Last.fm

In [314]:
# Generic Function for sending requests

def send_request(user=username, page=1, limit=200, key=key1, method='user.getrecenttracks'):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':user,
    'page': page,
    'api_key': key,
    'method': method,
    'format': 'json'
    }
    
    api_reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return api_reply

In [315]:
# Fetch a page and clean data before return.
# Also handles exceptions.

def get_page(num=1, username=username, key=key1, method='user.getrecenttracks'):
    
    try:
        response = send_request(page=num, key=key, method=method)
        
        if response.status_code != 200:
            raise Exception(api_reply.text)
        
        page = (response.json())['recenttracks']['track']
        
        if num == 1: 
            global totalPages
            totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
        
        if (num!=1) & ('@attr' in page[0].keys()): 
            if (page[0]['@attr']=={'nowplaying': 'true'}):
                del page[0]
        
        return page

    except:
        raise Exception("request status {}".format(response.status_code))

# Get the total number of pages for 200 scrobbles per page.

def get_totalPages(method='user.getrecenttracks'):
    response = send_request(method = method)
    totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
    return totalPages

totalPages = get_totalPages()

In [316]:
# Picks a key from given list of keys
def key_picker(num, ls):
    index_num = num % len(ls)
    return ls[index_num]

In [317]:
# Fetches all the pages and merges them into a list "pages"
# Uses a fancy progress bar to show progress.
# Large accounts may time time to fetch!


from IPython.core.display import clear_output

pages = []

for n in range(1, (totalPages)+1):
    
    print('fetched page {}/{}'.format(n, totalPages))
    
    spaces = (((totalPages-n)/totalPages)*50)
    bar = ((n/totalPages)*50)
    string = '[{}{}] {}% Completed'.format('~'*int(bar),' '*int(spaces), int(bar)*2)
    print(string)
    
    clear_output(wait=True)
    
    ky = key_picker(num = n, ls = key_list)
    fetched_page = get_page(num=n, key=ky)
    
    pages = [*pages, *fetched_page]
print()
print("Fetched {} pages, {} scrobbles".format(totalPages, len(pages)))


Fetched 68 pages, 13407 scrobbles


# Cleaning up Fetched Data with #Pandas

In [318]:
df = pd.DataFrame(pages)
df.head()

,artist,streamable,image,mbid,album,name,@attr,url,date
0,{'mbid': 'c07f0676-9143-4217-8a9f-4c26bd636f13...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",1c6f9d2a-132b-3837-b9c9-52d6489d7000,{'mbid': '44a48c19-f9b7-4fc7-a229-00fd344d4d18...,The Light Behind Your Eyes,{'nowplaying': 'true'},https://www.last.fm/music/My+Chemical+Romance/...,NaN
1,"{'mbid': '', '#text': 'Virtual Riot'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Simulation (Deluxe Vers...",REDLINE - color bass doggo VIP,NaN,https://www.last.fm/music/Virtual+Riot/_/REDLI...,"{'uts': '1645355370', '#text': '20 Feb 2022, 1..."
2,{'mbid': '33db34de-5419-4d4c-a45b-3dfd6b7d8a72...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",4375aa86-9378-4f82-b4c2-b7524b32ae68,{'mbid': 'b8f73c40-8565-4c0e-8f67-cef3df8e8660...,REDLINE,NaN,https://www.last.fm/music/Virtual+Riot/_/REDLINE,"{'uts': '1645355195', '#text': '20 Feb 2022, 1..."
3,"{'mbid': '', '#text': 'Virtual Riot'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Simulation (Deluxe Vers...",REDLINE - color bass doggo VIP,NaN,https://www.last.fm/music/Virtual+Riot/_/REDLI...,"{'uts': '1645355019', '#text': '20 Feb 2022, 1..."
4,{'mbid': '33db34de-5419-4d4c-a45b-3dfd6b7d8a72...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",4375aa86-9378-4f82-b4c2-b7524b32ae68,{'mbid': 'b8f73c40-8565-4c0e-8f67-cef3df8e8660...,REDLINE,NaN,https://www.last.fm/music/Virtual+Riot/_/REDLINE,"{'uts': '1645354844', '#text': '20 Feb 2022, 1..."


In [319]:
df.drop(['image','streamable'], inplace=True, axis=1)
if '@attr' in df.columns:
  df.drop('@attr', inplace=True, axis = 1)
df.head()

,artist,mbid,album,name,url,date
0,{'mbid': 'c07f0676-9143-4217-8a9f-4c26bd636f13...,1c6f9d2a-132b-3837-b9c9-52d6489d7000,{'mbid': '44a48c19-f9b7-4fc7-a229-00fd344d4d18...,The Light Behind Your Eyes,https://www.last.fm/music/My+Chemical+Romance/...,NaN
1,"{'mbid': '', '#text': 'Virtual Riot'}",,"{'mbid': '', '#text': 'Simulation (Deluxe Vers...",REDLINE - color bass doggo VIP,https://www.last.fm/music/Virtual+Riot/_/REDLI...,"{'uts': '1645355370', '#text': '20 Feb 2022, 1..."
2,{'mbid': '33db34de-5419-4d4c-a45b-3dfd6b7d8a72...,4375aa86-9378-4f82-b4c2-b7524b32ae68,{'mbid': 'b8f73c40-8565-4c0e-8f67-cef3df8e8660...,REDLINE,https://www.last.fm/music/Virtual+Riot/_/REDLINE,"{'uts': '1645355195', '#text': '20 Feb 2022, 1..."
3,"{'mbid': '', '#text': 'Virtual Riot'}",,"{'mbid': '', '#text': 'Simulation (Deluxe Vers...",REDLINE - color bass doggo VIP,https://www.last.fm/music/Virtual+Riot/_/REDLI...,"{'uts': '1645355019', '#text': '20 Feb 2022, 1..."
4,{'mbid': '33db34de-5419-4d4c-a45b-3dfd6b7d8a72...,4375aa86-9378-4f82-b4c2-b7524b32ae68,{'mbid': 'b8f73c40-8565-4c0e-8f67-cef3df8e8660...,REDLINE,https://www.last.fm/music/Virtual+Riot/_/REDLINE,"{'uts': '1645354844', '#text': '20 Feb 2022, 1..."


In [320]:
#Split the artist column, and rename the df. Drop the orginal column
artist_split = df['artist'].apply(pd.Series).rename(columns = {'mbid':'artist_mbid','#text':'artist'})
df.drop(columns = ['artist'], inplace=True)

#Split the album column, and rename the df. Drop the original column
album_split = df['album'].apply(pd.Series).rename(columns = {'mbid':'album_mbid','#text':'album'})
df.drop(columns = ['album'], inplace=True)

#Split the date column, and rename the df. Drop the original column
#Also check if residual '0' column is made. (If the top song is being streamed, it wont include the timing.)
date_split = df['date'].apply(pd.Series).rename(columns = {'uts':'uts','#text':'date'})
df.drop(columns = ['date'], inplace=True)
if 0 in date_split.columns:
    date_split.drop(0, axis=1, inplace=True)

#Merging the new columns into main df
df = pd.concat([df, artist_split, album_split, date_split],axis=1)
df.head()

,mbid,name,url,artist_mbid,artist,album_mbid,album,date,uts
0,1c6f9d2a-132b-3837-b9c9-52d6489d7000,The Light Behind Your Eyes,https://www.last.fm/music/My+Chemical+Romance/...,c07f0676-9143-4217-8a9f-4c26bd636f13,My Chemical Romance,44a48c19-f9b7-4fc7-a229-00fd344d4d18,Number Three,NaN,NaN
1,,REDLINE - color bass doggo VIP,https://www.last.fm/music/Virtual+Riot/_/REDLI...,,Virtual Riot,,Simulation (Deluxe Version),"20 Feb 2022, 11:09",1645355370
2,4375aa86-9378-4f82-b4c2-b7524b32ae68,REDLINE,https://www.last.fm/music/Virtual+Riot/_/REDLINE,33db34de-5419-4d4c-a45b-3dfd6b7d8a72,Virtual Riot,b8f73c40-8565-4c0e-8f67-cef3df8e8660,SIMULATION,"20 Feb 2022, 11:06",1645355195
3,,REDLINE - color bass doggo VIP,https://www.last.fm/music/Virtual+Riot/_/REDLI...,,Virtual Riot,,Simulation (Deluxe Version),"20 Feb 2022, 11:03",1645355019
4,4375aa86-9378-4f82-b4c2-b7524b32ae68,REDLINE,https://www.last.fm/music/Virtual+Riot/_/REDLINE,33db34de-5419-4d4c-a45b-3dfd6b7d8a72,Virtual Riot,b8f73c40-8565-4c0e-8f67-cef3df8e8660,SIMULATION,"20 Feb 2022, 11:00",1645354844


In [321]:
#Reordering columns
col_reorder = ['artist','name','album','date','artist_mbid','mbid','album_mbid','uts','url']
df = df.reindex(col_reorder, axis=1)
df.head()

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
0,My Chemical Romance,The Light Behind Your Eyes,Number Three,NaN,c07f0676-9143-4217-8a9f-4c26bd636f13,1c6f9d2a-132b-3837-b9c9-52d6489d7000,44a48c19-f9b7-4fc7-a229-00fd344d4d18,NaN,https://www.last.fm/music/My+Chemical+Romance/...
1,Virtual Riot,REDLINE - color bass doggo VIP,Simulation (Deluxe Version),"20 Feb 2022, 11:09",,,,1645355370,https://www.last.fm/music/Virtual+Riot/_/REDLI...
2,Virtual Riot,REDLINE,SIMULATION,"20 Feb 2022, 11:06",33db34de-5419-4d4c-a45b-3dfd6b7d8a72,4375aa86-9378-4f82-b4c2-b7524b32ae68,b8f73c40-8565-4c0e-8f67-cef3df8e8660,1645355195,https://www.last.fm/music/Virtual+Riot/_/REDLINE
3,Virtual Riot,REDLINE - color bass doggo VIP,Simulation (Deluxe Version),"20 Feb 2022, 11:03",,,,1645355019,https://www.last.fm/music/Virtual+Riot/_/REDLI...
4,Virtual Riot,REDLINE,SIMULATION,"20 Feb 2022, 11:00",33db34de-5419-4d4c-a45b-3dfd6b7d8a72,4375aa86-9378-4f82-b4c2-b7524b32ae68,b8f73c40-8565-4c0e-8f67-cef3df8e8660,1645354844,https://www.last.fm/music/Virtual+Riot/_/REDLINE


# Exporting into CSV

In [322]:
# df.to_csv('export.csv', encoding='utf-8')
# df.to_json('export.json', encoding='utf-8')

# Data Analysis & Visualization

In [323]:
# Finding Top Songs:
TopSongs = pd.DataFrame(df['name'].value_counts())
TopSongs

,name
Fu re te Fu re ru,52
Signal,50
us,50
contrast,41
だから僕は音楽を辞めた,40
...,...
Ezio’s Family - Origins Version,1
Prosperity and Decay,1
The Streets,1
"Run, Shay! Run!",1


In [324]:
TopArtists = pd.DataFrame(df['artist'].value_counts())
TopArtists.head(10)

,artist
Yutaka Yamada,822
TK from Ling tosite sigure,530
Ling Tosite Sigure,426
ヨルシカ,308
Aimer,306
TK from 凛として時雨,296
Hiroyuki Sawano,291
小田朋美,242
amazarashi,223
RADWIMPS,179


# Working with Timeseries Data

In [325]:
# 20 Feb 2022, 10:03
import datetime as dt
df['date'] = pd.to_datetime(df['date'])

try:
  df['date'] = (df['date']).dt.tz_localize('UTC')
except:
  pass

try:
  # df['date'] = (df['date']).dt.tz_convert('UTC')
  pass
except:
  pass

df['date'] = df['date'] + dt.timedelta(5,30,00)

In [338]:
map1 = (df['date'].dt.time > dt.time(00,00,00))
map2 = (df['date'].dt.time < dt.time(23,59,00))
map3 = (df['date'].dt.date == dt.date(2022,2,22))
df[map3]
# df

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
103,麦吉_Maggie,耳わほう,耳わほう,2022-02-22 15:34:30+00:00,,,,1645112093,https://www.last.fm/music/%E9%BA%A6%E5%90%89_M...
104,BUMP OF CHICKEN,Gravity,Gravity,2022-02-22 15:34:30+00:00,0f718079-e5ea-4cfb-b512-b2d04da66901,1b65b724-1417-4656-a838-05fb5a8bccc6,d14e2455-751f-4280-87dc-adbd5563c457,1645112093,https://www.last.fm/music/BUMP+OF+CHICKEN/_/Gr...
105,Martin Garrix,Scared to Be Lonely,"Scared To Be Lonely by Dua Lipa, Martin Garrix",2022-02-22 13:28:30+00:00,3e1f2ee4-16be-4406-bf18-6173840cf2b1,4a22fdf9-3778-4609-aaed-c944682556f2,,1645104505,https://www.last.fm/music/Martin+Garrix/_/Scar...
106,Faithless,Insomnia - Radio edit,Pure... 90s Dance Party,2022-02-22 13:24:30+00:00,23d9d74d-c95e-46a6-be26-a6075c49747a,,,1645104285,https://www.last.fm/music/Faithless/_/Insomnia...
107,Flume,Spring,Hi This Is Flume (Mixtape),2022-02-22 13:21:30+00:00,,,,1645104068,https://www.last.fm/music/Flume/_/Spring
108,Burial,Fostercare,5: Five Years of Hyperdub,2022-02-22 13:15:30+00:00,9ddce51c-2b75-4b3e-ac8c-1db09e7c89c6,26e0598b-aa25-4b66-8ee3-1ef06d0dcf87,,1645103722,https://www.last.fm/music/Burial/_/Fostercare
109,Milet,Fly High,Visions,2022-02-22 13:06:30+00:00,906c39c0-e839-43db-a644-ea1a0101e8f8,60b7a083-c856-40a0-a050-7245dc8db323,be70f035-fd11-4b94-8cb9-85019d0c7a20,1645103213,https://www.last.fm/music/Milet/_/Fly+High
110,Blake Shelton,Minimum Wage,Body Language,2022-02-22 13:03:30+00:00,2ad66473-ccb3-41d2-8756-b56680cdfe62,0045136b-b776-4b56-8d28-d77e54c0fa01,1e598721-cf98-466a-a0f6-fe35c74af2be,1645102985,https://www.last.fm/music/Blake+Shelton/_/Mini...
111,Luke Combs,What You See Is What You Get,What You See Is What You Get,2022-02-22 13:00:30+00:00,c20ee61f-071f-4e65-9c81-45ee931a54ce,0d18efa7-777a-4490-bb03-f1818a4e23ff,0a839c3e-848b-4cf3-8ffa-90afd1cafe41,1645102812,https://www.last.fm/music/Luke+Combs/_/What+Yo...
112,Luke Combs,Doin' This,Doin' This,2022-02-22 12:56:30+00:00,c20ee61f-071f-4e65-9c81-45ee931a54ce,ba306695-4d45-4cb9-9cb8-c81ff1a29163,21f0d56f-c0cb-467a-9fcd-d6b846e60351,1645102566,https://www.last.fm/music/Luke+Combs/_/Doin%27...
